In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
# uncomment this to read and process all excel files again

def raw_tennis_data_from_excel():

    df = pd.DataFrame()
    for year in range(2001, 2022):
        
        file = ""
        if year < 2013:
            file = f"{year}.xls"
            print(file)
        else:
            assert(year >= 2013)
            file = f"{year}.xlsx"
            print(file)
        
        year_df = pd.read_excel(file)
        year_df['Year'] = year
        df = df.append(year_df)

    return df

# uncomment this to read in raw data again
# tennis_df = raw_tennis_data_from_excel()

In [ ]:
# print(len(tennis_df))
# tennis_df.head()

In [ ]:
# saves df as file so we don't need to load all the excel files again
# uncomment this to read in raw data again
# tennis_df.to_pickle("tennis_df_raw.pkl")


tennis_df = pd.read_pickle("tennis_df_raw.pkl")


In [ ]:
print(len(tennis_df))
assert(len(tennis_df)==55604)

In [ ]:
tennis_df.head()

In [ ]:
# only want to consider data with completed matches
print(len(tennis_df))
tennis_df = tennis_df[tennis_df['Comment']=='Completed']
print(len(tennis_df))

In [ ]:
# as of right now, don't see these columns as adding much to the model/data
# once we can get this to work with BET365 then we can look at other books
cols_list_drop = ['Best of',
       'Comment', 'CBW', 'CBL', 'GBW', 'GBL', 'IWW', 'IWL', 'SBW', 'SBL',
        'B&WW', 'B&WL', 'EXW', 'EXL', 'PSW', 'PSL',
       'WPts', 'LPts', 'UBW', 'UBL', 'LBW', 'LBL', 'SJW', 'SJL', 'MaxW',
       'MaxL', 'AvgW', 'AvgL']

In [ ]:
tennis_df = tennis_df.drop(cols_list_drop, axis=1)

In [ ]:
tennis_df.head()

In [ ]:
tennis_df.head()

In [ ]:
tennis_df.columns

In [ ]:
# dropping all na's because the only NA's were in B365 and in rank - can't run model without either of these
tennis_df = tennis_df.dropna(
    subset=['ATP', 'Location', 'Tournament', 'Date', 'Series', 
    'Court', 'Surface', 'Round', 'Winner', 'Loser', 'WRank', 
    'LRank', 'Wsets', 'Lsets', 'Year', 'B365W', 'B365L']
    )

In [ ]:
# make sure that the drop na worked
percent_missing = tennis_df.isnull().sum() * 100 / len(tennis_df)
missing_value_df = pd.DataFrame({'column_name': tennis_df.columns,
                             'percent_missing': percent_missing})
display(missing_value_df)

In [ ]:
len(tennis_df)

In [ ]:
tennis_df.reset_index(inplace=True, drop=True)

In [ ]:
tennis_df.head()

In [ ]:
# convert ranks to ints
tennis_df['WRank'] = tennis_df['WRank'].astype(int)
tennis_df['LRank'] = tennis_df['LRank'].astype(int)

In [ ]:
tennis_df.dtypes

In [ ]:
# this code identifies which tournaments don't have semifinals or finals

unique_tournaments = tennis_df['Tournament'].unique()
unique_rounds = tennis_df['Round'].unique()
display(unique_rounds)

def find_no_rounds_available(tournaments_list, round_list):

    for tournament in tournaments_list:

        tournament_df = tennis_df[tennis_df['Tournament']==tournament]

        for round_num in round_list:

            round_df = tournament_df[tournament_df['Round']==round_num]

            if len(round_df)==0:
                print(tournament, f"NO {round_num}")


find_no_rounds_available(
    tournaments_list=unique_tournaments, 
    round_list=['Semifinals', 'The Final'])

In [ ]:
queens_club = tennis_df[tennis_df['Tournament']=='Queens Club']
# display('Queens Club', queens_club)
display("Queens club", queens_club['Round'].unique())
zhuhai = tennis_df[tennis_df['Tournament']=='Zhuhai Open']
# display('Zhuhai Open', zhuhai)
display("Zhuhai", zhuhai['Round'].unique())

In [ ]:
tennis_df[tennis_df['Tournament']=='Zhuhai Open'].head()

In [ ]:
# dropping both of these tournaments from our set because they don't have both a semifinals and a finals
# we need both of these to measure how much pressure exists in a match

dropped_tournaments = ['Queens club', 'Zhuhai Open']

tennis_df = tennis_df[~tennis_df['Tournament'].isin(dropped_tournaments)]

assert(len(tennis_df[tennis_df['Tournament']=='Zhuhai Open'])==0)
assert(len(tennis_df[tennis_df['Tournament']=='Queens club'])==0)

In [ ]:
# ranks final as highest pressure
# ranks semifinal as next highest
# categorizes everything else as not pressure

def tournament_rank_helper(round_tn):

    round_rank = None
    if round_tn == "The Final":
        round_rank = 1
    elif round_tn == "2nd Round":
        round_rank = 2
    else:
        round_rank = 3

    return round_rank


tennis_df['Tournament Rank'] = tennis_df['Round'].apply(tournament_rank_helper)


In [ ]:
tennis_df.head()

In [ ]:
display(tennis_df['Round'].unique())
display(tennis_df['Tournament Rank'].unique())

In [ ]:
# convert European odds into probabilities
# B365 factors in the VIM (how much the book makes)

tennis_df['B365W_prob'] = (1/tennis_df['B365W']).round(3)
tennis_df['B365L_prob'] = (1/tennis_df['B365L']).round(3)
tennis_df['B365_total_prob'] = tennis_df['B365W_prob'] + tennis_df['B365L_prob']
tennis_df['B365_diff_from_1'] = tennis_df['B365_total_prob'] - 1

# This takes VIM out to just get the real probabilities
tennis_df['real_prob_W'] = tennis_df['B365W_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_prob_L'] = tennis_df['B365L_prob'] - (tennis_df['B365_diff_from_1']/2)
tennis_df['real_total_prob'] = tennis_df['real_prob_W'] + tennis_df['real_prob_L']
tennis_df['real_diff_from_1'] = tennis_df['real_total_prob'] - 1

In [ ]:
tennis_df.head(5)

In [ ]:
drop_cols = [ 'Round', 'Year', 'B365W', 'B365L', 'B365W_prob', 'B365L_prob', 'B365_total_prob', 'B365_diff_from_1', 'real_total_prob', 'real_diff_from_1']

tennis_df = tennis_df.drop(drop_cols, axis=1)

In [ ]:
import datetime

In [ ]:
# creates player dictionary with basic info
def player_dict_generator(name, rank, prob):
    
    player_dict = {}
    player_dict['name'] = name
    player_dict['rank'] = rank
    player_dict['prob'] = prob
    
    return player_dict

In [ ]:
# this creates a dictionary based on the season
# Dec, Jan, Feb are the '0' season (Winter)
# March, April, May are the '1' season (Spring) etc.
season_dict = dict.fromkeys([12, 1, 2], 0)
season_dict.update(dict.fromkeys([3, 4, 5], 1))
season_dict.update(dict.fromkeys([6, 7, 8], 2))
season_dict.update(dict.fromkeys([9, 10, 11], 3))
season_dict

In [ ]:
tennis_df.head()

In [ ]:
# concerts columns W1 through L5 to float columns
# this makes it so we can fill NAs and then add game points horizontally
for i in range(1,6):
    win_col = "W"+str(i)
    loss_col = "L"+str(i)

    # dataframe[column] = dataframe[column].replace(r'\s+', np.nan, regex=True)
    
    tennis_df[win_col] = tennis_df[win_col].replace(r'\s+', 0, regex=True)
    tennis_df[loss_col] = tennis_df[loss_col].replace(r'\s+', 0, regex=True)

    tennis_df[win_col] = tennis_df[win_col].astype(float)
    tennis_df[loss_col] = tennis_df[loss_col].astype(float)

In [ ]:
tennis_df.head()

In [ ]:
# fills NaNs with 0s for the W1 through L5 columns
def fill_nans_with_zero(df):

    # display missing vals cols
    print("Before Fill NA:")
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                'percent_missing': percent_missing})
    display(missing_value_df)

    values = {
    "W1": 0, 
    "L1": 0, 
    "W2": 0, 
    "L2": 0,
    "W3": 0,
    "L3": 0,
    "W4": 0,
    "L4": 0,
    "W5": 0,
    "L5": 0,
    }
    df = df.fillna(value=values)
    # df = df.fillna(0)

    print("After Fill NA:")
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                'percent_missing': percent_missing})
    display(missing_value_df)

    return df



In [ ]:
tennis_df = fill_nans_with_zero(tennis_df)

In [ ]:
# check to make sure we're working with good datatypes
tennis_df.dtypes

Here is where we are able to extract the winner and loser information into a higher and lower seed. This is important because we are ultimately going be predicting the higher seed's chance at victory, but with the dataframe before this code block this could not be possible. This code block also calculates the amount of games won/lost by both the higher and lower seeds as well as the sets won/lost

In [ ]:
def tennis_df_manipulation_helper(row):

    # set up information about the winner and loser

    # winner information
    winner = player_dict_generator(
        name=row['Winner'], 
        rank=row['WRank'], 
        prob=row['real_prob_W'])

    # loser information
    loser = player_dict_generator(
        name=row['Loser'], 
        rank=row['LRank'], 
        prob=row['real_prob_L'])

    
    # use comparisons to figure out the higher and lower rank
    higher_rank = {}
    lower_rank = {}
    if winner['rank'] < loser['rank']:
        higher_rank = winner
        lower_rank = loser
    else:
        higher_rank = loser
        lower_rank = winner

    # return the new higher and lower ranked values back into the row
    row['HR_name'] = higher_rank['name']
    row['HR_rank'] = higher_rank['rank']
    row['HR_prob'] = higher_rank['prob']

    row['LR_name'] = lower_rank['name']
    row['LR_rank'] = lower_rank['rank']
    row['LR_prob'] = lower_rank['prob']


    # now we calculate how many games and sets the higher and lower seed won/lost in the match
    hr_sets_won = None
    lr_sets_won = None
    hr_sets_lost = None
    lr_sets_lost = None

    hr_games_won = None
    lr_games_won = None
    hr_games_lost = None
    lr_games_lost = None

    winner_games_won = row['W1']+row['W2']+row['W3']+row['W4']+row['W5']
    loser_games_won = row['L1']+row['L2']+row['L3']+row['L4']+row['L5']
    # the winner lost the amount of games won by the winner, and viseversa
    winner_games_lost = loser_games_won
    loser_games_lost = winner_games_won

    if higher_rank['name']==winner['name']:
        assert(higher_rank['name']==winner['name'])
        assert(lower_rank['name']==loser['name'])
        # this happens if the higher ranked seed won the match
        hr_sets_won = row['Wsets']
        hr_sets_lost = row['Lsets']
        lr_sets_won = row['Lsets']
        lr_sets_lost = row['Wsets']

        hr_games_won = winner_games_won
        hr_games_lost = winner_games_lost

        lr_games_won = loser_games_won
        lr_games_lost = loser_games_lost


    else:
        assert(higher_rank['name']==loser['name'])
        assert(lower_rank['name']==winner['name'])
        # this happens if the lower ranked seed won the match
        hr_sets_won = row['Lsets']
        hr_sets_lost = row['Wsets']
        lr_sets_won = row['Wsets']
        lr_sets_lost = row['Lsets']

        hr_games_won = loser_games_won
        hr_games_lost = loser_games_lost

        lr_games_won = winner_games_won
        lr_games_lost = winner_games_lost

    
    # assign values back to the row
    row['HR_Wsets'] = hr_sets_won
    row['LR_Wsets'] = lr_sets_won
    row['HR_Lsets'] = hr_sets_lost
    row['LR_Lsets'] = lr_sets_lost

    row['HR_Wgames'] = hr_games_won
    row['LR_Wgames'] = lr_games_won
    row['HR_Lgames'] = hr_games_lost
    row['LR_Lgames'] = lr_games_lost

    # calculate the time of year of the match (winter, summer, etc.)
    date_of_match = row['Date']
    match_month = date_of_match.month

    row['Match_szn'] = season_dict[match_month]

    return row

In [ ]:
tennis_df = tennis_df.apply(tennis_df_manipulation_helper, axis=1)

In [ ]:
tennis_df.head()

In [ ]:
tennis_df.columns

In [ ]:
# now that we know how many games in the match each player won/lost, we can drop this information
drop_cols = [ 'W1', 'L1', 'W2', 'L2', 'W3', 'L3',
       'W4', 'L4', 'W5', 'L5']

tennis_df = tennis_df.drop(drop_cols, axis=1)

In [ ]:
tennis_df.head()

Starting here, we start with our feature engineering. The feature engineering will be done by extracting certain data from the current tennis_df dataset, and adding it to a new dataset that will be suitable to run ML on. The first step in doing this is to create a deep copy of the datafame in its current stage. This is a historical reference we will be querying into to extract information previous to any given match we are analyzing (we can only look into the past, looking into the future would be data leakage). tennis_df_historical will be our historical reference dataframe

In [ ]:
tennis_df_historical = tennis_df.copy()

Starting here, we have our helper functions. All of these layers of helper functions stem off of the ML_df_function, which returns a ML-readable dataframe given the tennis_df dataframe

In [ ]:
# calculates the difference of set wins a player has historically had, and returns this as a rate of matches the player has played in
# for example, if set_dif_per_match is 2, this could mean that over the span of 2 matches, the player could have won 6
# sets and lost 2 sets. (6-2) = 4, 4/2(for the 2 matches)=2.0
# essentially it is calculating the margin of victory via set totals
def generate_set_information(df, player_name):
    sets_won = 0
    sets_lost = 0

    # making sure we are only looking at dataframes containing
    # out player of interest
    assert((
        len(df[df['LR_name']==player_name]) +
        len(df[df['HR_name']==player_name]))==
        len(df)
        )
    # print(player_name, "History:")
    # display(df)

    # query matches where playr was higher rank, then add up sets won and lost
    higher_rank_matches = df[df['HR_name']==player_name]
    sets_won += sum(higher_rank_matches['HR_Wsets'])
    sets_lost += sum(higher_rank_matches['HR_Lsets'])

    # display(higher_rank_matches)

    # query matche where player was the lower rank, then add up sets won and lost
    lower_rank_matches = df[df['LR_name']==player_name]
    sets_won += sum(lower_rank_matches['LR_Wsets'])
    sets_lost += sum(lower_rank_matches['LR_Lsets'])

    # display(lower_rank_matches)

    # calculate set difference, total matches in dataframe
    set_difference = sets_won - sets_lost
    total_matches = len(df)

    # try calculating the set margin of victory, if not posible (such as
    # a divide by 0 error if there are no matches), then return NaN
    try:
        set_dif_per_match = set_difference/total_matches
    except:
        set_dif_per_match = np.nan

    return set_dif_per_match



In [ ]:
# uses same logic as generate_set_information, except this time we are calculating metrics for 
# total games played
def generate_game_information(df, player_name):
    games_won = 0
    games_lost = 0

    # making sure we are only looking at dataframes containing
    # out player of interest
    assert((
        len(df[df['LR_name']==player_name]) +
        len(df[df['HR_name']==player_name]))==
        len(df)
        )
    # print(player_name, "History:")
    # display(df)

    # query matches where player was higher rank, then add up games won and lost
    higher_rank_matches = df[df['HR_name']==player_name]
    games_won += sum(higher_rank_matches['HR_Wgames'])
    games_lost += sum(higher_rank_matches['HR_Lgames'])

    # display(higher_rank_matches)
    # query matches where player was lower rank, then add up games won and lost
    lower_rank_matches = df[df['LR_name']==player_name]
    games_won += sum(lower_rank_matches['LR_Wgames'])
    games_lost += sum(lower_rank_matches['LR_Lgames'])

    # display(lower_rank_matches)

    # calculate difference in games, and calculate total matches
    games_difference = games_won - games_lost
    total_matches = len(df)


    # try calculating the game margin of victory, if not posible (such as
    # a divide by 0 error if there are no matches), then return NaN
    try:
        games_dif_per_match = games_difference/total_matches
    except:
        games_dif_per_match = np.nan

    return games_dif_per_match

In [ ]:
# queries into whole dataset to filter so we're only looking at matches of a player before the match of interest
# this function is crucial to ensure we are not time traveling (data leakage)

def historical_match_data(player_name, date_of_match):
#     isolated all matches a player has played in (whole dataset)
    player_rank_history = tennis_df_historical.loc[(tennis_df_historical['Winner'] == player_name) | (tennis_df_historical['Loser'] == player_name)]

#     filter this to before the date of the match so we're only looking at historical matches
    player_rank_history = player_rank_history.loc[player_rank_history['Date'] < date_of_match]

    return player_rank_history

In [ ]:
# helper function is currently not used:

# returns weighted wins with the following formula:
# Rank = Wins * Wins / (Wins + Losses)
# https://stackoverflow.com/questions/7509219/weighted-win-percentage-by-number-of-games-played
def convert_wl_to_win_ratio(df, player_name):
    
    winning_matches = len(df.loc[df['Winner']==player_name])
    losing_matches = len(df.loc[df['Loser']==player_name])
    total_matches = winning_matches+losing_matches
    assert(winning_matches + losing_matches == len(df))

    win_win_ratio = (winning_matches*winning_matches)/total_matches
    return win_win_ratio

In [ ]:
# helper function is currently not used:

# returns weighted wins with the following formula:
# Rank = (Wins + constant * Average Win % of all players) / (Wins + Losses + constant)
# https://stackoverflow.com/questions/7509219/weighted-win-percentage-by-number-of-games-played

def convert_wl_to_win_ratio(df, player_name, constant, av_win_pct):
    
    winning_matches = len(df.loc[df['Winner']==player_name])
    losing_matches = len(df.loc[df['Loser']==player_name])
    total_matches = winning_matches+losing_matches
    assert(winning_matches + losing_matches == len(df))

    ratio = ((winning_matches+constant)*av_win_pct)/(winning_matches+losing_matches+constant)

    return ratio

In [ ]:
# calculates number historic championships
def number_historic_championchips(df, player_name):

    # Tournament Rank == 1 means final round match
    final_matches = df.loc[df['Tournament Rank']==1]
    winner_final_match = final_matches.loc[final_matches['Winner']==player_name]
    num_championchips = len(winner_final_match)

    # print("HISTORICAL CHAMPIONCHIPS")
    # display(winner_final_match)
    # print(num_championchips)
    
    return num_championchips

In [ ]:
# returns winning percentage for given df
# this function should only be called after historical_match_data(), otherwise it is data leakage
def winning_percentage(player_df, player_name):
    
#     find where player wins, loses, then get percentage
    appears_winner = len(player_df.loc[player_df['Winner']==player_name])
    appears_loser = len(player_df.loc[player_df['Loser']==player_name])
    assert(appears_winner + appears_loser == len(player_df))
    winning_percentage = appears_winner/len(player_df)
    
    return winning_percentage

In [ ]:
# returns multiple winning percentages: based on court type (outdoor, indoor), surface (grass, clay, etc), and series (Grand slam, etc.)
def winning_percentage_stats(player_history, player_name, court_type, 
surface_type, series_type, season_type, tournament_type):

#     isolate court type
    player_court_type = player_history.loc[player_history['Court']==court_type]
# try to calculate winning percentage based on court type, if not possible return NaN
    try: court_type_winning_pct = winning_percentage(
        player_df=player_court_type, 
        player_name=player_name)
    except:
        assert(len(player_court_type)==0)
        court_type_winning_pct = np.nan

    
#     repeat same as above for surface, series, season, tournament
    player_surface_type = player_history.loc[player_history['Surface']==surface_type]
    try: surface_type_winning_pct = winning_percentage(
        player_df=player_surface_type, 
        player_name=player_name)
    except:
        assert(len(player_surface_type)==0)
        surface_type_winning_pct = np.nan


    player_series_type = player_history.loc[player_history['Series']==series_type]
    try:
        series_type_winning_pct = winning_percentage(
        player_df=player_series_type, 
        player_name=player_name)
    except:
        assert(len(player_series_type)==0)
        series_type_winning_pct = np.nan


    player_season_type = player_history.loc[player_history['Match_szn']==season_type]
    try:
        szn_type_winning_pct = winning_percentage(
        player_df=player_season_type, 
        player_name=player_name)
    except:
        assert(len(player_season_type)==0)
        szn_type_winning_pct = np.nan


    player_tournament = player_history.loc[player_history['Tournament']==tournament_type]
    try: tournament_win_pct = winning_percentage(
        player_df=player_tournament, 
        player_name=player_name)
    except:
        assert(len(player_tournament)==0)
        tournament_win_pct = np.nan

    
       # print(player_name, court_type, surface_type, series_type)
    #     display(player_court_type)
    #     display(player_surface_type)
    #     display(player_series_type)

    #     return results as a tuple
    return (court_type_winning_pct, 
        surface_type_winning_pct, 
        series_type_winning_pct, 
        szn_type_winning_pct,
        tournament_win_pct)

In [ ]:
# find matches when they play head to head, assess performance based on games won
def head_to_head_performance(player_history, player_name, opponent_name):
    
    # isolate head to head matchups
    head_to_head_matches = player_history[
        (player_history['HR_name']==opponent_name) | 
        (player_history['LR_name']==opponent_name)]

    # if players never played each other, return NaN
    if len(head_to_head_matches) < 1:
        # print("No matches")
        return np.nan
    

# takes the difference of amount of games won vs lost, then divides by num matches
# calculates the average margin of victory (terms of number of games) per match
    player_head_to_head_games_pct = generate_game_information(
        df=head_to_head_matches, 
        player_name=player_name)

    # print("Head to head:")
    # display(head_to_head_matches)
    # print("win pct:", win_pct_h2h)
    return player_head_to_head_games_pct

    

In [ ]:
# this looks at performance as higher or lower seed
def performance_as_rank(df, player_name, rank_comparison):

    # display("player history", df)


    results_df = None
    assert(rank_comparison=='higher' or rank_comparison=='lower')
    if rank_comparison=='higher':

        # isloate matches where the player was the higher rank
        results_df = df[df['HR_name']==player_name]

        # if there were no matches where player was higher seed, return NaN
        if len(results_df)==0:
            return np.nan
        else:
            # make sure the datagrame only contains info with the higher ranked player
            assert(len(results_df['HR_name'].unique())==1)
    else:
        assert(rank_comparison=='lower')

        # isolate matches where player was lower ranked 
        results_df = df[df['LR_name']==player_name]
        # if there was no matches where player was lower rank, return NaN
        if len(results_df)==0:
            return np.nan
        else:
            # make sure df only has info on player as lower rank
            assert(len(results_df['LR_name'].unique())==1)
            

# calculates the average margin of victory (terms of number of games) per match
    rank_game_dif_pct = generate_game_information(
        df=results_df, 
        player_name=player_name)

    return rank_game_dif_pct
    


In [ ]:
# returns a bunch of stats for a player in a match
def player_stats(name, rank_comparison, opponent_name, rank, prob, last_n_games, last_n_days, match_date, 
court_type, surface_type, series_type, season_type, tournament):
    
#     get player historical data, this is crucial for generating rest of stats
    player_history = historical_match_data(
        player_name=name,
        date_of_match=match_date
    )

    # if player has less than 5 matches, don't count in dataset
    if len(player_history) < 5:
        return None
        
    else:

        # winning percentage of all time before match date
        all_time_win_pct = winning_percentage(
            player_df=player_history,
            player_name=name)

        # average margin of victory (in terms of sets) historically
        sets_dif_per_match = generate_set_information(
            df=player_history, 
            player_name=name)

        # average margin of victory (in therms of games) historically
        games_dif_per_match =  generate_game_information(
            df=player_history, 
            player_name=name)

        # calculates historical performance of the player (if they are the higher seed, this returns past performance
        # (in terms of sets_dif_per_match) as the higher seed, and if they are the lower seed, vice versa)
        rank_performance_game_dif = performance_as_rank(
            df=player_history, 
            player_name=name, 
            rank_comparison=rank_comparison)

        # number of historic championchips won by the player
        num_champs = number_historic_championchips(
            df=player_history, 
            player_name=name)

        # head to head performance against opponent (measured in games_dif_per_match)
        h2h_games_dif_per_match = head_to_head_performance(
            player_history=player_history, 
            player_name=name, 
            opponent_name=opponent_name)

        # amount of career matches
        career_matches = len(player_history)

        # calculates the last n days before a match 
        # if match is on Dec 31, and last_n_days is 30,
        # then n_days_before_match is Dec 1
        last_n_days_dt = datetime.timedelta(last_n_days)
        n_days_before_match = match_date - last_n_days_dt

        # this isolates match data so only data within n_days_before_match
        # of the match is included 
        # (following with the example above, this isolates matches between Dec 1 and Dec 30)
        player_historical_match_days = player_history.loc[
            (player_history['Date'] < match_date) & 
            ((player_history['Date'] > n_days_before_match))
        ]

        # calculates amount of matches played in n_days_before_match
        # this is meant to calculate for player usage rate
        # a we would expect a player who has been playing 1 match every 2 days for the last year to be 
        # more tired than a player playing 1 match every 10 days for the last year, the code below calculates that
        amount_matches_n_days_before = len(player_historical_match_days)
        match_per_day_pecentage = amount_matches_n_days_before/last_n_days
        

        # find player last n games (to calculate win percentage for recent matches)
        player_last_n_games = player_history[-last_n_games:]

        # get recent player win pct (win pct in the last n matches)
        recent_win_pct = winning_percentage(
            player_df=player_last_n_games,
            player_name=name)

        # calculte set and game difference for recent matches as well
        recent_sets_dif_per_match = generate_set_information(
            df=player_last_n_games, 
            player_name=name)

        recent_games_dif_per_match =  generate_game_information(
            df=player_last_n_games, 
            player_name=name)


    #     get player win percentage on the court the match is on, the sruface type of the match, and the series type
        (player_court_type_win_pct, 
        player_surface_type_win_pct, 
        player_series_type_win_pct, 
        szn_type_winning_pct, 
        tournament_win_pct) = winning_percentage_stats(
            player_history=player_history, 
            player_name=name, 
            court_type=court_type,
            surface_type=surface_type,
            series_type=series_type,
            season_type=season_type,
            tournament_type=tournament,
        )

    #     return all this info for the player in a big dictionary:
        ret_dict = {
                'name':name,
                'rank':rank,
                'prob':prob,

                'sets_dif_per_match':sets_dif_per_match,
                'games_dif_per_match': games_dif_per_match,
                'rank_performance_game_dif':rank_performance_game_dif,
                
                'num_champs':num_champs,
                # number finals appearances?
                'h2h_games_dif_per_match':h2h_games_dif_per_match,
                'career_matches':career_matches,
                'matches_per_day_rate': match_per_day_pecentage,
                'all_time_win_pct':all_time_win_pct,

                # totals within last n games (as of right now it is 20)
                'recent_win_pct': recent_win_pct,
                'recent_sets_dif_per_match': recent_sets_dif_per_match,
                'recent_games_dif_per_match':recent_games_dif_per_match,


                # all of these seem optional at the moment (correlated very high with all time win%)
                'court_type_win_pct':player_court_type_win_pct,
                'surface_type_win_pct':player_surface_type_win_pct,
                'series_type_win_pct':player_series_type_win_pct,
                'szn_type_win_pct':szn_type_winning_pct,
                'tournament_win_pct':tournament_win_pct,
            }

        return ret_dict

In [ ]:
from datetime import date

# this creates the dataset we can use for ML, driver function for feature engineering

def ML_df_function(df, start_row=None, end_row=None):

#     set up return DF with our new formatted data
#     to find what columns to put for final_df:
#     display(returned_row.keys())
    final_columns = ['Location', 'Date', 'Series', 'Court', 'Surface', 'Szn', 'ATP', 'HR_win?', 'HR_name', 'LR_name', 
    'higher_ranked_rank', 'higher_ranked_prob', 'higher_ranked_sets_dif_per_match', 'higher_ranked_games_dif_per_match',
     'higher_ranked_rank_performance_game_dif', 'higher_ranked_num_champs', 'higher_ranked_h2h_games_dif_per_match', 
     'higher_ranked_career_matches', 'higher_ranked_matches_per_day_rate', 'higher_ranked_all_time_win_pct', 
     'higher_ranked_recent_win_pct', 'higher_ranked_recent_sets_dif_per_match', 'higher_ranked_recent_games_dif_per_match', 
     'higher_ranked_court_type_win_pct', 'higher_ranked_surface_type_win_pct', 'higher_ranked_series_type_win_pct', 
     'higher_ranked_szn_type_win_pct', 'higher_ranked_tournament_win_pct', 'lower_ranked_rank', 'lower_ranked_prob', 
     'lower_ranked_sets_dif_per_match', 'lower_ranked_games_dif_per_match', 'lower_ranked_rank_performance_game_dif',
      'lower_ranked_num_champs', 'lower_ranked_h2h_games_dif_per_match', 'lower_ranked_career_matches', 'lower_ranked_matches_per_day_rate',
       'lower_ranked_all_time_win_pct', 'lower_ranked_recent_win_pct', 'lower_ranked_recent_sets_dif_per_match', 
       'lower_ranked_recent_games_dif_per_match', 'lower_ranked_court_type_win_pct', 'lower_ranked_surface_type_win_pct',
        'lower_ranked_series_type_win_pct', 'lower_ranked_szn_type_win_pct', 'lower_ranked_tournament_win_pct', 'diff_rank',
         'diff_prob', 'diff_sets_dif_per_match', 'diff_games_dif_per_match', 'diff_rank_performance_game_dif', 'diff_num_champs',
          'diff_h2h_games_dif_per_match', 'diff_career_matches', 'diff_matches_per_day_rate', 'diff_all_time_win_pct', 
          'diff_recent_win_pct', 'diff_recent_sets_dif_per_match', 'diff_recent_games_dif_per_match', 'diff_court_type_win_pct',
           'diff_surface_type_win_pct', 'diff_series_type_win_pct', 'diff_szn_type_win_pct', 'diff_tournament_win_pct']
    
    # create final dataframe
    final_df = pd.DataFrame(columns = final_columns)

    # code below tells us the progress of processing the data
    num_rows = end_row-start_row
    print(f"{num_rows} total rows in dataset")
    current_row = 0


    for i in range(start_row,end_row):

        current_row += 1
        if (current_row % 5000==0):
            print(f"{current_row} rows processed.")


        # create dictionary to return the new row
        # return location, date, series, court, etc.
        returned_row = {}
        returned_row['Location'] = df.loc[i,'Location']
        returned_row['Date'] = df.loc[i,'Date']
        returned_row['Series'] = df.loc[i,'Series']
        returned_row['Court'] = df.loc[i,'Court']
        returned_row['Surface'] = df.loc[i,'Surface']
        returned_row['Szn'] = df.loc[i,'Match_szn']
        returned_row['ATP'] = df.loc[i,'ATP']


#         DISPLAY original df row
        # display(df.iloc[i].to_frame().T)
        # display(df.loc[i,'HR_name'])

#         get date of the match
        date_of_match = df.loc[i,'Date']

        # previous games to be considered for the recency statistics
        last_n_games = 20

        # days previous to match to calculate usage rate before match
        last_n_days = 180
        
        # calculate all the stats for the higher ranked player
        higher_ranked_player_stats = player_stats(
            name=df.loc[i,'HR_name'],
            rank_comparison = 'higher',
            opponent_name=df.loc[i,'LR_name'],
            rank=df.loc[i,'HR_rank'],
            prob=df.loc[i,'HR_prob'],
            last_n_games=last_n_games,
            last_n_days=last_n_days,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series'], 
            season_type=df.loc[i,'Match_szn'], 
            tournament=df.loc[i,'Tournament']
        )

        # calculate all the stats for the lower ranked player
        lower_ranked_player_stats = player_stats(
            name=df.loc[i,'LR_name'],
            rank_comparison = 'lower',
            opponent_name=df.loc[i,'HR_name'],
            rank=df.loc[i,'LR_rank'],
            prob=df.loc[i,'LR_prob'],
            last_n_games=last_n_games,
            last_n_days=last_n_days,
            match_date = df.loc[i,'Date'],
            court_type=df.loc[i,'Court'],
            surface_type=df.loc[i,'Surface'],
            series_type=df.loc[i,'Series'],
            season_type=df.loc[i,'Match_szn'], 
            tournament=df.loc[i,'Tournament']
        )

        # if the minimum games threshold (5 as of last writing) hasn't been met, go to the next match
        if (higher_ranked_player_stats == None) or (lower_ranked_player_stats == None):
            i += 1
        else:
            # display(higher_ranked_player_stats)
            # display(lower_ranked_player_stats)

            # for each stat, append this to the returned row (row to be added to new ML df)

            # calculate if higher ranked player won
            hr_name = higher_ranked_player_stats['name']
            # print(hr_name)
            winner = df.loc[i,'Winner']
            # print(winner)

            if hr_name == winner:
                returned_row['HR_win?']=1
            else:
                returned_row['HR_win?']=0


            # save name to returned row and then delete name so we are only taking diff on numeric columns
            returned_row['HR_name'] = higher_ranked_player_stats['name']
            returned_row['LR_name'] = lower_ranked_player_stats['name']

            del higher_ranked_player_stats['name']
            del lower_ranked_player_stats['name']


            # get the keys and values for the higher ranked and lower ranked players
            value_keys = list(higher_ranked_player_stats.keys())
            high_rank_list = list(higher_ranked_player_stats.values())
            low_rank_list = list(lower_ranked_player_stats.values())

            # add all the stats to returned row
            for key, stat in zip(value_keys, high_rank_list):
                returned_row[f'higher_ranked_{key}'] = stat

            for key, stat in zip(value_keys, low_rank_list):
                returned_row[f'lower_ranked_{key}'] = stat

            # display(high_rank_list)
            # display(low_rank_list)
            
            # now take the difference of these values (important in seeing how much of 
            # an advantage the higher ranked player has over the lesser ranked)
            diffs = [a_i - b_i for a_i, b_i in zip(high_rank_list, low_rank_list)]
            # print("diffs")
            # display(diffs)


            # add all these diff values to the returned row for ML
            for key, diff in zip(value_keys, diffs):
                returned_row[f'diff_{key}'] = diff


            # uncomment this to find out what the keys of returned_row are, and 
            # then we can make these the new columns for the dataframe:
            
            # display(returned_row.keys())
            final_df = final_df.append(returned_row, ignore_index=True)
    return final_df


In [ ]:
# important to reset the index so the counter works properly
tennis_df = tennis_df.reset_index(drop=True)

In [ ]:
tennis_df.tail()

In [ ]:
# runs the driver function above, expect this to take at least 2 hours

ML_dataframe = ML_df_function(
    df=tennis_df, 
    start_row=0, end_row=len(tennis_df)
    # start_row=40000, end_row=40010
)


# Features:
#     How much rank is chancged before match

In [ ]:
ML_dataframe.tail()

In [ ]:
# check where missing vals are in cols:
pd.set_option("display.max_rows", None)
percent_missing = ML_dataframe.isnull().sum() * 100 / len(ML_dataframe)
missing_value_df = pd.DataFrame({'column_name': ML_dataframe.columns,
                            'percent_missing': percent_missing})
display(missing_value_df)

In [ ]:
# this moves our target variable, HR_win? to the end of the dataframe
HR_win_col = ML_dataframe.pop('HR_win?')
ML_dataframe.insert(len(ML_dataframe.columns), 'HR_win?', HR_win_col)

In [ ]:
ML_dataframe.head()

In [ ]:
# save dataframe to csv
ML_dataframe.to_csv("complete_data.csv")